https://incidecoder.com/ingredients

https://incidecoder.com/products

In [12]:
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd

# Skincare brands to scrape product data from
- Kiehl's 
- Clarins
- Sulwhasoo
- Hera
- La Mer
- Clinique
- Lancome
- Elizabeth Arden


In [13]:
brands = ["Kiehl's", "Clarins", "Sulwhasoo", "Hera", "La Mer", "Clinique", "Lancome", "Elizabeth Arden","Skin Inc", "Decorte", "Fresh", "Sk ii", "Sum 37", "Givenchy", "Loreal", "Guerlain","Vichy","The Ordinary"]
brands_with_plus = [brand.replace(" ", "+") for brand in brands]
brands = [brand.replace("'","").replace(" ", "-") for brand in brands]
brands = [brand.lower() for brand in brands]
print(brands)


['kiehls', 'clarins', 'sulwhasoo', 'hera', 'la-mer', 'clinique', 'lancome', 'elizabeth-arden', 'skin-inc', 'decorte', 'fresh', 'sk-ii', 'sum-37', 'givenchy', 'loreal', 'guerlain', 'vichy', 'the-ordinary']


In [14]:
from bs4 import BeautifulSoup

def scrape_product_details(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')

    # Extracting product name
    product_name = soup.find('h1').get_text(strip=True) if soup.find('h1') else 'Unknown'

    # Extracting brief description
    description = soup.find('span', class_='product-details').get_text(strip=True) if soup.find('span', class_='product-details') else 'Description not available'

    # Extracting ingredients overview
    ingredients = soup.find('div', class_='ingredients-overview').get_text(strip=True) if soup.find('div', class_='ingredients-overview') else 'Ingredients not listed'

    # Extracting Key Ingredients from the Highlights section
    key_ingredients = {}
    function_text = ""
    highlights_section = soup.find('div', class_='ingred-list-by-function-block')  # Adjust class name as necessary
    if highlights_section:
        for function in highlights_section.find_all('div', class_='highlight-category'):  # Adjust as necessary
            function_text = soup.find('a', class_='func-link').get_text(strip=True) if function.find('a') else 'Unknown Function'
            # Find all <a> tags with class 'ingred-link' and extract text
            key_ingredients = [tag.get_text(strip=True) for tag in soup.find_all('a', class_='ingred-link')]
            print(function_text)
            print(key_ingredients)

    # Extracting Ingredients Function
    ingredients_function = {}
    function_section = soup.find('div', class_='ingredients-function-section')  # Adjust class name as necessary
    if function_section:
        for item in function_section.find_all('div', class_='ingredient-item'):  # Adjust as necessary
            ingredient_name = item.find('h3').get_text(strip=True) if item.find('h3') else 'Unknown Ingredient'
            function = item.find('p').get_text(strip=True) if item.find('p') else 'Function not listed'
            ingredients_function[ingredient_name] = function

    return {
        'product_name': product_name,
        'description': description,
        'ingredients': ingredients,
        'key_ingredients': key_ingredients,
        'ingredients_function': function_text
    }


In [15]:
# Function to clean and remove duplicates from ingredient list
def clean_ingredients(ingredient_list):
    cleaned_list = []
    seen = set()
    for ingredient in ingredient_list:
        # Normalize the ingredient string
        normalized_ingredient = ingredient.replace('\u200b', '/').replace('/', ', ').replace(', ,',',')
        if normalized_ingredient not in seen:
            seen.add(normalized_ingredient)
            cleaned_list.append(normalized_ingredient)
    return cleaned_list

In [16]:
from urllib.parse import urljoin

In [17]:

def scrape_product_details(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')

    # Extracting product summary (assuming it's within a <p> tag with a specific class, adjust as necessary)
    product_summary = soup.find('p', class_='product-summary').get_text(strip=True) if soup.find('p', class_='product-summary') else 'Summary not available'

    # Extracting Key Ingredients and their functions
    ingredients_info = {}
    for block in soup.find_all("div", class_="ingredlist-by-function-block"):
        category_name = block.find('h3').get_text(strip=True) if block.find('h3') else 'Unknown Category'
        ingredients = []
        for ingred_span in block.find_all('span'):
            ingredient_link = ingred_span.find('a', class_='ingred-link')
            if ingredient_link:
                ingredient_name = ingredient_link.get_text(strip=True)
                ingredient_function = ingred_span.find_previous('span', class_='bold').get_text(strip=True) if ingred_span.find_previous('span', class_='bold') else 'Unknown Function'
                ingredients.append((ingredient_name, ingredient_function))
        ingredients_info[category_name] = ingredients

    return {
        'summary': product_summary,
        'ingredients_info': ingredients_info
    }


In [18]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin

def scrape_incidecoder(brand):
    base_url = 'https://incidecoder.com'
    search_url = base_url + '/search?query='+brand # only clarins products for now
    print(search_url)
    headers = {'User-Agent': 'Mozilla/5.0'}

    all_product_details = []
    all_product_names = []
    all_product_urls = []
    all_product_summaries = []

    while search_url:
        response = requests.get(search_url, headers=headers)
        # print("here1")
        if response.status_code == 200:
            # print("here2")
            soup = BeautifulSoup(response.content, 'html.parser')
            products_div = soup.find('div', class_='paddingbl')
            if products_div:
                # print("here3")
                links = products_div.find_all('a')
                pdt_urls = [urljoin(base_url, link['href']) for link in links]
                pdt_names = [link.text.strip() for link in links]

                # Append product names and URLs to their respective lists
                all_product_names.extend(pdt_names)
                all_product_urls.extend(pdt_urls)
                # print(all_product_names)
                for pdt_url in pdt_urls:
                    pdt_response = requests.get(pdt_url, headers=headers)
                    if pdt_response.status_code == 200:
                        summary_and_product_details = scrape_product_details(pdt_response.content)
                        product_details = summary_and_product_details['ingredients_info']
                        product_summary = summary_and_product_details['summary']
                        all_product_details.append(product_details)
                        all_product_summaries.append(product_summary)
                    else:
                        print(f'Failed to retrieve product details for {pdt_url}')


                # Find the next page link
                next_page_link = soup.find('a', string="Next page >>")
                if next_page_link:
                    next_page_url = next_page_link.get('href')
                    search_url = urljoin(base_url, next_page_url)
                else:
                    break  # No more next page, exit the loop
            else:
                print('No product div found on the page.')
                break
        else:
            print('Failed to retrieve the webpage')
            break

    all_product_names = all_product_names[:-1]
    all_product_urls = all_product_urls[:-1]
    all_product_details = all_product_details[:-1]
    all_product_summaries = all_product_summaries[:-1]
    return all_product_names, all_product_urls, all_product_summaries, all_product_details

# # Usage
# pdt_names, pdt_urls, pdt_summaries, product_ingredients_details = scrape_incidecoder()
# print(pdt_names, pdt_urls, product_ingredients_details)


In [19]:
# brands = ["Kiehl's", "Clarins", "Sulwhasoo", "Hera", "La Mer", "Clinique", "Lancome", "Elizabeth Arden","Skin Inc", "Decorte", "Fresh", "Sk ii", "Sum 37", "Givenchy", "Loreal", "Guerlain","Vichy","The Ordinary"]
# brands = ['clarins']

In [20]:
info_dict = {}
for brand in brands:
    info_dict[brand] = scrape_incidecoder(brand)
    

https://incidecoder.com/search?query=kiehls


https://incidecoder.com/search?query=clarins
https://incidecoder.com/search?query=sulwhasoo
https://incidecoder.com/search?query=hera
https://incidecoder.com/search?query=la-mer
https://incidecoder.com/search?query=clinique
https://incidecoder.com/search?query=lancome
https://incidecoder.com/search?query=elizabeth-arden
https://incidecoder.com/search?query=skin-inc
https://incidecoder.com/search?query=decorte
https://incidecoder.com/search?query=fresh
https://incidecoder.com/search?query=sk-ii
https://incidecoder.com/search?query=sum-37
https://incidecoder.com/search?query=givenchy
https://incidecoder.com/search?query=loreal
https://incidecoder.com/search?query=guerlain
https://incidecoder.com/search?query=vichy
https://incidecoder.com/search?query=the-ordinary


Takes about 70 minutes to scrape product data from these 18 brands

In [21]:
print(info_dict.keys())

dict_keys(['kiehls', 'clarins', 'sulwhasoo', 'hera', 'la-mer', 'clinique', 'lancome', 'elizabeth-arden', 'skin-inc', 'decorte', 'fresh', 'sk-ii', 'sum-37', 'givenchy', 'loreal', 'guerlain', 'vichy', 'the-ordinary'])


In [22]:
# # all_product_names, all_product_urls, all_product_summaries, all_product_details
# info_dict['sulwhasoo'][0] # all product names
# info_dict['sulwhasoo'][1] # all product urls
# # info_dict['hera'][2] # all product summaries # TO DO; currently getting "Summary not available" for all items :( )
# info_dict['clarins'][3] # all product details

In [23]:
info_dict['sulwhasoo']

(['Sulwhasoo Bloomstay Vitalizing Water',
  'Sulwhasoo Snowise Brightening Exfoliating Mask',
  'Sulwhasoo Snowise Brightening Serum',
  'Sulwhasoo Essential Revitalizing Serum Ex',
  'Sulwhasoo Essential Balancing Emulsion',
  'Sulwhasoo Capsulized Ginseng Fortifying Serum',
  'Sulwhasoo Timetreasure Invigorating Eye Serum',
  'Sulwhasoo Snowise Brightening Emulsion',
  'Sulwhasoo Essential Balancing Water Ex',
  'Sulwhasoo Radiance Energy Mask',
  'Sulwhasoo Clarifying Mask',
  'Sulwhasoo Everefine Lifting Ampoule Serum',
  'Sulwhasoo Snowise Brightening Spot Serum',
  'Sulwhasoo Concentrated Ginseng Renewing Serum',
  'Sulwhasoo Concentrated Ginseng Renewing Cream Ex',
  'Sulwhasoo Essential Refining Eye Cream',
  'Sulwhasoo Men Relaxing Uv Protector Spf50+ Pa+++',
  'Sulwhasoo Bloomstay Vitalizing Cream',
  'Sulwhasoo Concentrated Ginseng Renewing Water',
  'Sulwhasoo Perfecting Cushion Brightening',
  'Sulwhasoo Perfecting Cushion Ex',
  'Sulwhasoo Bloomstay Vitalizing Serum',
  '

In [24]:
# print(len(pdt_names))
# print(len(pdt_urls))
# print(len(pdt_summaries))
# print(len(product_ingredients_details))
# in future to add product instructions (from different data source)

# To get general function from the most common function among all constituent ingredients in a product
- Note: In future, to change the logic here because this may not hold true for all products; i.e. The most common function among all constituent ingredients of a product may not be the product's main function. The most common function may depend on other factors such as ingredient concentration.
- Could be a better idea to change this part: overall function is a list of all the functions of the active ingredients in the product

In [25]:
from collections import Counter
 
def most_frequent(List):
    occurence_count = Counter(List)
    return occurence_count.most_common(1)[0][0]

In [26]:
# iterating through every brand
df = pd.DataFrame()

for key, value in info_dict.items():
    # brand = key
    pdt_names = value[0]
    pdt_urls = value[1]
    pdt_summaries = value[2]
    product_ingredients_details = value[3]
    
    for product in product_ingredients_details:
        # Combine the two lists and assign to 'All Ingredients'
        product['All Ingredients'] = product.get('Key Ingredients', []) + product.get('Other Ingredients', [])
        
    prods_ingred_lists = []
    prods_ingred_functions_lists = []
    ingred_functions_dict = {}
    for i in range(len(product_ingredients_details)): # for each product
        prod_ingred_list = []
        prod_ingred_function_list = []
        for j in range(len(product_ingredients_details[i]['All Ingredients'])):
            key = product_ingredients_details[i]['All Ingredients'][j][0]
            val = product_ingredients_details[i]['All Ingredients'][j][1]
            prod_ingred_list.append(key)
            prod_ingred_function_list.append(val)
            
            ingred_functions_dict[key] = val
            
            
            
        prod_ingred_list = clean_ingredients(prod_ingred_list)
        prods_ingred_lists.append(prod_ingred_list)
        prods_ingred_functions_lists.append(prod_ingred_function_list)
        
        overall_function_list = []
        for function_list in prods_ingred_functions_lists:
            # print(function_list)
            if function_list:
                most_common_function= most_frequent(function_list)
            else:
                most_common_function = ""
            overall_function_list.append(most_common_function)
    
    products_table = pd.DataFrame({'Name': pdt_names,
                            'Overall Function': overall_function_list,
                            'Ingredients': prods_ingred_lists})
    
    df = pd.concat([df,products_table], ignore_index=True)

    

In [27]:
print(df.head())

                                              Name       Overall Function  \
0           Kiehl’s Vital Skin Strengthening Serum              Perfuming   
1  Kiehl’s Ultra Facial Overnight Hydrating Masque  Moisturizer/humectant   
2    Kiehl’s Clearly Corrective Dark Spot Solution  Moisturizer/humectant   
3          Kiehl’s Ultra Facial Oil-Free Gel-Cream  Moisturizer/humectant   
4                                 Kiehl’s Eye Fuel           Preservative   

                                         Ingredients  
0  [Tocopherol, Panax Ginseng Root Extract, Rosma...  
1  [Glycerin, Squalane, Urea, Pseudoalteromonas F...  
2  [Salicylic Acid, 3-O-Ethyl Ascorbic Acid, Toco...  
3  [Glycerin, Sodium Hyaluronate, Pseudoalteromon...  
4  [Niacinamide, Caffeine, Tocopherol, Ascorbyl G...  


In [28]:
print(len(df))


3956


In [32]:
print(df.columns)

Index(['Name', 'Overall Function', 'Ingredients'], dtype='object')


In [33]:
df.to_csv("many_products.csv", index=False)

In [35]:
len(df)

3956